# **4 Kernel Power on SVM and Regularized Logistic Regression (Ray)**

Generating a dataset with 100 samples and 5,000,000 features from the standard normal distribution.

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
data_raw = pd.read_csv('spectf.csv')
data_train, data_test = train_test_split(data_raw, test_size=0.20, random_state=0)
print(data_raw)

     1.0  59.0  52.0  70.0  67.0  73.0  66.0  72.0  61.0  58.0  ...  66.0.3  \
0    1.0  72.0  62.0  69.0  67.0  78.0  82.0  74.0  65.0  69.0  ...    65.0   
1    1.0  71.0  62.0  70.0  64.0  67.0  64.0  79.0  65.0  70.0  ...    73.0   
2    1.0  69.0  71.0  70.0  78.0  61.0  63.0  67.0  65.0  59.0  ...    61.0   
3    1.0  70.0  66.0  61.0  66.0  61.0  58.0  69.0  69.0  72.0  ...    67.0   
4    1.0  57.0  69.0  68.0  75.0  69.0  74.0  73.0  71.0  57.0  ...    63.0   
..   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...     ...   
261  0.0  74.0  69.0  75.0  70.0  70.0  74.0  77.0  77.0  65.0  ...    66.0   
262  0.0  72.0  61.0  64.0  66.0  64.0  59.0  68.0  66.0  76.0  ...    69.0   
263  0.0  75.0  73.0  72.0  77.0  68.0  67.0  76.0  73.0  67.0  ...    70.0   
264  0.0  59.0  62.0  72.0  74.0  66.0  66.0  74.0  76.0  63.0  ...    65.0   
265  0.0  64.0  66.0  68.0  71.0  62.0  64.0  74.0  73.0  63.0  ...    70.0   

     56.0.1  62.0  56.0.2  72.0.3  62.0.1  74.0.2  

**4.1**

In [ ]:
import numpy as np